## Inactive vs. Active Accounts
---

As Dan suggested, the refitted model penalizes inactive account much LESS than the old one, which may be the cause behind the performance lift.

Let's take a look how do the models measure up against one another.

Maybe we should keep the customer model fitting option if it outperforms on the active accounts.

Let's define activeness potentially as ` (df_dev.nr_past_transactions > 0) & (df_dev.nr_transactions_30d > 0)`

Maybe just `df_dev.nr_past_transactions > 0`


In [20]:
import sys, os, json
sys.path.insert(1, "../../")
from src.combine import combine_data
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
import src.monitoring.refit as refit
import src.monitoring.governance as gvrn
import src.monitoring.monitoring as mntr
import src.monitoring.utils as mu

%matplotlib inline
plt.style.use('seaborn')

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load test data and models

In [ ]:
# load modeling_df
dev_path = "../../data/combined_all_features/combined_1616138792.feather"
df = pd.read_feather(dev_path)
df = mu.preprocess(df)

In [ ]:
start = pd.to_datetime("2019-01-01")
end = pd.to_datetime("2020-11-30")  # 1 m
modeling_df = df[df.sample_date.between(start, end)]
valid_df_dec = df[(df.is_static==True) &
              (df.sample_date==pd.to_datetime("2020-12-01"))]

### Refitted Model
---

In [ ]:
# prev model
prev_model_s3_key = "jxu/money-risk-models/customer-risk-model/models/customer_risk_target_no_giact_time_since_last_link.pkl"
old_model = mu.read_pickle_from_s3("sofi-data-science", prev_model_s3_key)

new_model_booster_w_ind = lgb.Booster(model_file="artifacts/model_20200630.txt")
new_model_booster = lgb.Booster(model_file="artifacts/model_20201130.txt")

In [ ]:
# old model
valid_df_dec["pred_20200101"] = old_model.predict_proba(valid_df_dec[old_model.feature_name_])[:,1]
valid_df_dec["score_20200101"] = mu.scale_scores(valid_df_dec["pred_20200101"])

# new model with indeterminant
valid_df_dec["pred_200630"]

# new model without indeterminant
valid_df_dec["pred_20201130"] = new_model_booster.predict(valid_df_dec[old_model.feature_name_])
valid_df_dec["score_20201130"] = mu.scale_scores(valid_df_dec["pred_20201130"])